In [2]:
import csv

In [10]:
i=0
outfile = "haqae_data.txt"
with open("hm_indexed.csv", 'r') as handle:
    with open(outfile, 'w') as out:
        reader = csv.reader(handle)
        for line in reader:
            out.write(line[3] + " ")
            i += 1
            if i > 10:
                out.write("\n")
                i = 0

In [ ]:
# Create files:
<TUP> v, s, o, p ... v, s, o, p # <TUP> {} {} {} {} ##
<TUP> v_t, s_t, ...
4 tuples per line
#use null char for prep
fix text field for vocab in happydb

In [16]:

import torch
import torchtext.data as ttdata
import torchtext.datasets as ttdatasets
from torchtext import data

In [20]:
class SentenceDataset(ttdata.Dataset):
    'Data set which has a single sentence per line'

    def __init__(self, path, text_field, target_field , src_seq_length=50, min_seq_length=8, n_cloze=False, add_eos=True):

        """
        Args
            path (str) : Filename of text file with dataset
            vocab (Torchtext Vocab object)
            filter_pred (callable) : Only use examples for which filter_pred(example) is TRUE
        """
        #text_field = ExtendableField(vocab)

#         if add_eos:
#             target_field = ExtendableField(vocab, eos_token=EOS_TOK)
#         else:
#             target_field = ExtendableField(vocab) # added this for narrative cloze
       

        fields = [('text', text_field), ('target', target_field)]
        examples = []
        with open(path, 'r') as f:
            for line in f:
                text = line
                if n_cloze:
                    text = text.split("<TUP>") 
                    actual_event = text[-1] #last event
                    text = text[:-1] # ignore the last tuple
                    text = "<TUP>".join(text)
                    #print("cloze text is {}".format(text))
                    # text has t-1 events and target has the t event
                    examples.append(ttdata.Example.fromlist([text, actual_event], fields))
                else:
                    examples.append(ttdata.Example.fromlist([text, text], fields))

        def filter_pred(example):
            return len(example.text) <= src_seq_length and len(example.text) >= min_seq_length
 
        super(SentenceDataset, self).__init__(examples, fields, filter_pred=filter_pred)



In [22]:
text_field = data.Field(lower=True)
label_field = data.Field(sequential=False)
dataset = SentenceDataset("haqae_formated.txt", text_field, label_field , 50, add_eos=True)

In [23]:
text_field.build_vocab(dataset, vectors="glove.6B.100d")

.vector_cache/glove.6B.zip: 862MB [01:09, 8.10MB/s]                              
100%|█████████▉| 398645/400000 [00:16<00:00, 24139.46it/s]

In [26]:
import pickle
with open("vocab.pkl", "wb") as handle:
    pickle.dump(text_field.vocab, handle)

In [29]:
dir(text_field)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'batch_first',
 'build_vocab',
 'dtype',
 'dtypes',
 'eos_token',
 'fix_length',
 'include_lengths',
 'init_token',
 'is_target',
 'lower',
 'numericalize',
 'pad',
 'pad_first',
 'pad_token',
 'postprocessing',
 'preprocess',
 'preprocessing',
 'process',
 'sequential',
 'stop_words',
 'tokenize',
 'truncate_first',
 'unk_token',
 'use_vocab',
 'vocab',
 'vocab_cls']